#### Data analysis notebook

Reads in the data files and plot results

  * Instantiate analyzer class object **Geant4Analyzer(root_filename, label)**
  * Then apply cuts to the data
  * Plotting with the built-in function **plot_histogram()**
  * For user defined plots, just access the data elements directly: **Geant4Analyzer.data[element_name]**. Get the data element names from **Geant4Analyzer.data.keys()**

In [ ]:
from RunManager import RunManager
from Geant4Analyzer import Geant4Analyzer
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
manager = RunManager("../run/rundb.json")
display(manager.display_all_runs(include_deleted=False))

In [ ]:
# cut on the global event variables
cut = lambda data: (data['ncomp'] + data['nphot'] == 1) & (data['type'] == 0) & (data['e'] < 1200.) & (data['nclus'] == 1)
#cut = lambda data: (data['type'] == 0) & (data['e'] < 1200.) & (data['nclus'] == 1)

# additional cuts on the clusters
cut_hit = lambda data: (data['r'] < 1000.) & (np.abs(data['zh']) < 1000.) 

std = Geant4Analyzer("run_01", label="Geant4 standard", first_only=False)
std.preprocess_data(cut=cut, cut_hit=cut_hit)

fast = Geant4Analyzer("run_02", label="Accelerated MC", first_only=False)
fast.preprocess_data(cut=cut, cut_hit=cut_hit)

#fast2 = Geant4Analyzer("run_03", label="Accelerated MC", first_only=False)
#fast2.preprocess_data(cut=cut, cut_hit=cut_hit)


In [ ]:
nbin = 500
ax = std.plot_histogram('r', bins=nbin, range=(-1500,1500), show=False)
ax = fast.plot_histogram('r', bins=nbin, ax=ax, range=(-1500,1500), show=False)

plt.yscale('log')
plt.ylim(1e-5, 1e6)
plt.xlim(0, 1250)

plt.legend(frameon=False)
plt.show()


In [ ]:
plt.hist(std.data['nclus'], bins=10, range=(-0.5,9.5), histtype='step', label=std.label)

In [ ]:
plt.hist(std.data['r'])

In [ ]:
plt.hist(fast.data['nclus'], bins=10, range=(-0.5,9.5))
plt.hist(std.data['nclus'], bins=10, range=(-0.5,9.5))
plt.yscale('log')

In [ ]:
bins = 1000
h = plt.hist2d(std.data['xh'], std.data['yh'], bins=bins, range=((-1500, 1500), (-1500, 1500)))
circle = plt.Circle((0,0), std.settings['geometry_settings']['innerCryostatRadius'], color='w', fill=False, linewidth=1)
#plt.gca().add_artist(circle)
plt.show()

In [ ]:
from matplotlib.colors import LogNorm

bins = 100
h = plt.hist2d(std.data['r'], std.data['zh'], bins=bins, range=((0, 1500), (-1500, 1500)), norm=LogNorm())
plt.show()
h = plt.hist2d(fast.data['r'], fast.data['zh'], bins=bins, range=((0, 1500), (-1500, 1500)), norm=LogNorm())

plt.show()

In [ ]:
plt.hist(std.data['eh'], bins=100, range=(0, 1200), histtype='step', color='g', label='Geant4 standard')
plt.hist(fast.data['eh'], weights=np.exp(fast.data['wh']), bins=100, range=(0, 1200), histtype='step', color='b', label='Accelerated')

plt.ylim(1e-2, 1e4)
plt.yscale('log')
plt.show()



In [ ]:
plt.hist(std.data['nclus'][(std.data['ncomp']+std.data['nphot'])==1], bins=10, range=(-0.5,9.5), histtype='step', color='g', label='Geant4 standard')
plt.hist(fast.data['nclus'], weights=np.exp(fast.data['w']), bins=10, range=(-0.5,9.5), histtype='step', color='b', label='Accelerated')

plt.show()

In [ ]:
from RunManager import RunManager
from Geant4Analyzer import Geant4Analyzer
import matplotlib.pyplot as plt
import numpy as np

std_pencil  = Geant4Analyzer("run_11", first_only=False)
fast_pencil = Geant4Analyzer("run_10", first_only=False)

# fiducial volume definition
r_fid = std_pencil.settings['fiducialRadius']
z_fid = std_pencil.settings['fiducialHeight']/2.
e_cut = fast_pencil.settings['maxEnergy']

# cut on the global event variables
#cut = lambda data: (data['ncomp'] + data['nphot'] == 1) & (data['type'] == 0) & (data['e'] < 1200.) 
cut = lambda data: (data['nphot']==0) & (data['ncomp'] == 1) & (data['type'] == 0) & (data['e'] < e_cut) 

# additional cuts on the clusters
cut_hit = lambda data: (data['r'] < r_fid) & (np.abs(data['zh']) < z_fid) & (data['eh'] > 0) & (data['eh'] < 600)

std_pencil.preprocess_data(cut=cut, cut_hit=cut_hit)
fast_pencil.preprocess_data(cut=cut, cut_hit=cut_hit)

In [ ]:
nbin = 10
ax = std_pencil.plot_histogram('nclus', bins=nbin, range=(-0.5,9.5), show=False)
ax = fast_pencil.plot_histogram('nclus', bins=nbin, ax=ax, range=(-0.5,9.5), show=False)

plt.yscale('log')
#plt.ylim(1e-1, 1e5)
#plt.xlim(0, 1250)

plt.legend(frameon=False)
plt.show()

In [ ]:
nbin =100
ax = std_pencil.plot_histogram('e', bins=nbin, range=(0.,1100), show=False)
ax = fast_pencil.plot_histogram('e', bins=nbin, ax=ax, range=(0.,1000), show=False)
plt.yscale('log')
plt.ylim(1e-1, 1e5)
plt.show()

In [ ]:
plt.hist(std.data['e'][((std.data['ncomp']+std.data['nphot'])==1) & (std.data['type']==0)], bins=100, range=(-0.5,1100.5), histtype='step', color='g', label='Geant4 standard')
plt.hist(fast.data['e'], weights=np.exp(fast.data['w']), bins=100, range=(-0.5,1100.5), histtype='step', color='b', label='Accelerated')

plt.show()

In [ ]:
fast.data['w']